In [ ]:
import opt_note.scsp as scsp
import datetime
import didppy

In [ ]:
import marimo as mo
import nbformat

# `LA_SH` アルゴリズムのスピードを DIDP で上げたい

In [ ]:
def find_next_strategy(
    instance: list[str],
    chars: str,
    m: int,
) -> str:
    """
    現在の状態を受け取り, m 手進めたときに sum height が最大になる文字の選び方 (長さ m 以下の文字列として表される) を返す
    """

    dpmodel = didppy.Model(maximize=True, float_cost=False)

    instance_table = dpmodel.add_element_table(
        [[chars.index(c) for c in s] + [len(chars)] for s in instance]
    )

    index_types = [dpmodel.add_object_type(number=len(s) + 1) for s in instance]
    index_vars = [
        dpmodel.add_element_var(object_type=index_type, target=0)
        for index_type in index_types
    ]
    sol_len = dpmodel.add_int_var(target=0)

    dpmodel.add_base_case([sol_len == m])

    # 文字 char に従って進む
    for id_char, char in enumerate(chars):
        sum_height = sum(
            (instance_table[sidx, index_var] == id_char).if_then_else(1, 0)
            for sidx, index_var in enumerate(index_vars)
        )
        trans = didppy.Transition(
            name=f"{char}",
            cost=sum_height + didppy.IntExpr.state_cost(),
            effects=[
                (
                    index_var,
                    (instance_table[sidx, index_var] == id_char).if_then_else(
                        index_var + 1, index_var
                    ),
                )
                for sidx, index_var in enumerate(index_vars)
            ] + [
                (sol_len, sol_len + 1)
            ],
            preconditions=[sum_height > 0],
        )
        dpmodel.add_transition(trans)

    # Force transition
    end = didppy.Transition(
        name="",
        cost=didppy.IntExpr.state_cost(),
        effects=[(sol_len, m)],
        preconditions=[index_var == len(s) for s, index_var in zip(instance, index_vars)]
    )
    dpmodel.add_transition(end, forced=True)

    # Dual bound
    dual_bound_table = dpmodel.add_int_table(
        [
            [min(m, len(s) - idx) for idx in range(len(s) + 1)]
            for s in instance
        ]
    )
    bound = didppy.IntExpr(0)
    for sidx, index_var in enumerate(index_vars):
        bound += dual_bound_table[sidx, index_var]
    dpmodel.add_dual_bound(bound)

    dpsolver = didppy.CABS(
        dpmodel, threads=8, quiet=True
    )
    solution = dpsolver.search()

    print(f"step_time: {solution.time:f}")

    return "".join([trans.name for trans in solution.transitions])

In [ ]:
find_next_strategy(["ba", "cb", "ca", ""], "abc", 3)

step_time: 0.004687


In [ ]:
def solve(instance: list[str], m: int = 3, ll: int = 1) -> str:
    chars = "".join(sorted(list(set("".join(instance)))))
    state = tuple(0 for _ in instance)
    solution = ""

    count = 0
    while not all(idx == len(s) for idx, s in zip(state, instance)):
        next_str = find_next_strategy([s[idx:] for idx, s in zip(state, instance)], chars, m)
        if len(next_str) == 0:
            break
        count += 1
        # print(f"{count=}, {next_str=}")
        solution += next_str[:ll]
        for next_char in next_str[:ll]:
            state = tuple(
                idx + 1 if idx < len(s) and s[idx] == next_char else idx
                for idx, s in zip(state, instance)
            )

    return solution

In [ ]:
def bench1(instance: list[str], m = 3, l = 1) -> None:
    start = datetime.datetime.now()
    solution = scsp.model.la_sh.solve(instance, m, l)
    end = datetime.datetime.now()
    scsp.util.show(instance)
    scsp.util.show(instance, solution)
    print(f"wall time: {(end - start).total_seconds()}s")
    print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")

In [ ]:
def bench2(instance: list[str], m = 3, l = 1) -> None:
    start = datetime.datetime.now()
    solution = solve(instance, m, l)
    end = datetime.datetime.now()
    print()
    scsp.util.show(instance)
    scsp.util.show(instance, solution)
    print(f"wall time: {(end - start).total_seconds()}s")
    print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")

In [ ]:
bench1(scsp.example.load("uniform_q26n004k015-025.txt"))

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 70) ---
 Sol: itkgnekojiqfouhlcimpnbvaxxnhtqcgxycozgbrddbcsovxiorsuqzpplpvinngfssbxf
str1: -tkgn-k------uh---mp----x-nhtq-gx---z---------vxi--s------------------
str2: i------ojiqfo--l----nb--xx----c---------------v----suq-p---vi----ssbxf
str3: -------------u-lci--n------------yco--------sov--o----zpplp-----------
str4: i--g-e----------------va------------zgbrddbcs-v---r--------v-nngf-----

wall time: 0.007515s
solution is feasible: True


In [ ]:
bench2(scsp.example.load("uniform_q26n004k015-025.txt"))

step_time: 0.001013
step_time: 0.000993


step_time: 0.001013
step_time: 0.000924


step_time: 0.001018
step_time: 0.000951


step_time: 0.000947
step_time: 0.000940


step_time: 0.000967
step_time: 0.000990


...

step_time: 0.000757
step_time: 0.000795


step_time: 0.000733
step_time: 0.000916


step_time: 0.000904
step_time: 0.000747


step_time: 0.000741
step_time: 0.000415


step_time: 0.000110
step_time: 0.000052

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 68) ---
 Sol: itkgnekuhlcojimpxnhtqfycoslnbgxzoxcvaozxigpsubqrddbcspvirssvnblngxpf
str1: -tkgn-kuh-----mpxnhtq--------gxz---v---xi--s------------------------
str2: i----------oji------qf--o-lnb-x--xcv-------su-q------pvi-ss--b---x-f
str3: -------u-lc--i---n----ycos------o--v-oz---p----------p--------l---p-
str4: i--g-e-----------------------------va-z--g---b-rddbcs-v-r--vn--ng--f

wall time: 0.636509s
solution is feasible: True


In [ ]:
bench1(scsp.example.load("uniform_q26n008k015-025.txt"))

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 132) ---
 Sol: eingeojipbdcevaqdcrtkzfgjtbnkuhlcimpvxdnerddbycopflrhnzbsmorvigoplersbrowtxqgxckrdrlctodtmzprplpucpmqvxbbghinngfstdfuiqpvcdissbowdxf
str1: -------------------tk--g---nkuh---mp-x-n------------h--------------------t-qgx------------z----------vx----i----s-------------------
str2: -i---oji-------q------f------------------------o--l--n-b------------------x--xc----------------------v----------s---u-qpv--issb---xf
str3: -----------------------------u-lci-----n-----yco--------s-o-v--o--------------------------zp-plp------------------------------------
str4: -i-ge--------va------z-g--b--------------rddb-c---------s---v------r---------------------------------v------nngf--------------------
str5: 

In [ ]:
bench2(scsp.example.load("uniform_q26n008k015-025.txt"))

step_time: 0.001599


step_time: 0.001648


step_time: 0.001602


step_time: 0.001620


step_time: 0.002105


...

step_time: 0.000903


step_time: 0.000822


step_time: 0.000134


step_time: 0.000078


step_time: 0.000063

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 111) ---
 Sol: igepnbdcevazfojtdcrvxkgberydpfzdbulcinrsvzwxqkruvnfychmoplxsnbdromqvhiglctodztmprpxfqgxczvsuqlipxvicwdssebbhoxf
str1: ---------------t-----kg--------------n-------k-u-----hm-p-x-n-------h----t----------qgx-zv------x-i---s--------
str2: i------------oj---------------------i-------q-----f----o-l--nb--------------------x---xc-vsuq--p-vi---ss-b---xf
str3: ---------------------------------ulcin-------------yc--o---s----o--v------o-z--p-p-----------l-p---------------
str4: ige------vaz----------gb-r-d---db--c---sv-----r-vn----------n---------g------------f---------------------------
str5: ---p----------------------y-p-----l---r--z-x---u----c---p--------mqv--g--t-d-------f

In [ ]:
bench1(scsp.example.load("uniform_q26n016k015-025.txt"))

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 170) ---
  Sol: kpriflshtkqagenbkvxuwdgsfchxeqjafnvzfmpacdigqbcejtivwbahcokrdxerlcnvpfzksgjmiudbcrsvihtjodqbgkderwxzavntfmoybplngqrbpzvxucbbhefikoshmgopmqvgtowdfuivxcdvsbouqyzpplpvissbxf
str01: --------tk--g-n-k--u------h----------mp----------------------x----n------------------ht---q-g-----xz-v-----------------x-------i--s---------------------------------------
str02: ---i-----------------------------------------------------o----------------j-i-------------q-------------f-o---ln---b--

In [ ]:
bench2(scsp.example.load("uniform_q26n016k015-025.txt"))

step_time: 0.004948


step_time: 0.004655


step_time: 0.004794


step_time: 0.007304


step_time: 0.007811


...

step_time: 0.000977


step_time: 0.001020


step_time: 0.000175


step_time: 0.000168


step_time: 0.000072

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 156) ---
  Sol: kripxwuslxqhftkcgevaxjnfzpabdieusvdcgbqarvdnypfzhckodjiqbcesfmolvrzwxiuchozvnbpmgqjpwxanlctvfodkxgtdohmfkpskerzgqbcrtoqvigpxzsluivcwdqsjbopvxissaekgnbhwwxfy
str01: -------------tk-g-----n---------------------------k-------------------u-h------m---p-x-n-------------h--------------t-q--g-xz----v----------xis-------------
str02: --i------------------------------------------------o-jiq----f-ol------------nb-------x----------x-----------------c----v-----

In [ ]:
bench1(scsp.example.load("uniform_q05n010k010-010.txt"))

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 31) ---
  Sol: abdcbacdbeecdaebdecbadabcdebdde
str01: --dcb-c----cd--b--c-----c-e----
str02: -bd----dbee---e--e-b-d---------
str03: ---c-acd-eec--eb-e-------------
str04: a--------e--d---d----d---debdd-
str05: a--cb----eec-a-b--c-------e----
str06: -b--ba--be-----bd-cba----------
str07: -b--ba---e---aeb----ada--------
str08: ---------ee---e--ecb-d-b--e---e
str09: ---c--cd-ee-da--d-c--d---------
str10: -bd--a--b---d--b-e--a-a--d-----

wall time: 0.004176s
solution is feasible: True


In [ ]:
bench2(scsp.example.load("uniform_q05n010k010-010.txt"))

step_time: 0.001375
step_time: 0.001335
step_time: 0.003161


step_time: 0.002144
step_time: 0.003060


step_time: 0.001280
step_time: 0.001083
step_time: 0.001233


step_time: 0.001055
step_time: 0.003933


step_time: 0.001278
step_time: 0.001380
step_time: 0.001191


...

step_time: 0.002065
step_time: 0.001078
step_time: 0.000962


step_time: 0.001050
step_time: 0.000862
step_time: 0.000797


step_time: 0.000577
step_time: 0.000837


step_time: 0.000584
step_time: 0.000111
step_time: 0.000057


step_time: 0.000051

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 31) ---
  Sol: bdacbacdbeecdaeebadcbddeabcddee
str01: -d-cb-c----cd---b--c------c--e-
str02: bd-----dbee---eeb-d------------
str03: ---c-acd-eec--e-b------e-------
str04: --a------e--d-----d--dde-b-dd--
str05: --acb----eec-a--b--c---e-------
str06: b---ba--be------b-dcb---a------
str07: b---ba---e---ae-bad-----a------
str08: ---------ee---ee---cbd---b---ee
str09: ---c--cd-ee-da----dc-d---------
str10: bda-b--dbe---a---ad------------

wall time: 0.195555s
solution is feasible: True


In [ ]:
bench1(scsp.example.load("uniform_q05n050k010-010.txt"))

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 38) ---
  Sol: dacbedabcedabcdeaecbdacebad

In [ ]:
bench2(scsp.example.load("uniform_q05n050k010-010.txt"))

step_time: 0.002597


step_time: 0.003094


step_time: 0.002628


step_time: 0.002948


step_time: 0.002946


...

step_time: 0.001467


step_time: 0.001121


step_time: 0.000355


step_time: 0.000181


step_time: 0.000123

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 38) ---
  Sol: daecbd

In [ ]:
bench1(scsp.example.load("nucleotide_n010k010.txt"))

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 28) ---
  Sol: TATACACGGGATACGAATCGATCACAGA
str01: -AT----GGGATACG-------------
str02: -ATAC-C----T-----TC---C-C---
str03: ----CACG--A-A----T---T----GA
str04: TA-A-A----AT-C---T-G-T------
str05: -A-----GG--TA--A--C-A--A-A--
str06: T-T-C-C----TA-G----G-T-A----
str07: T-T----G---TA-GA-TC--T------
str08: T------GGGA-A-G--T---TC-----
str09: T-T-C-C---A--C-AA-C--T------
str10: T---C------TA--AA-CGA--A----

wall time: 0.002421s
solution is feasible: True


In [ ]:
bench2(scsp.example.load("nucleotide_n010k010.txt"))

step_time: 0.001063
step_time: 0.001184
step_time: 0.001070


step_time: 0.000973
step_time: 0.001031
step_time: 0.001095


step_time: 0.001249
step_time: 0.001018
step_time: 0.001018


step_time: 0.001009
step_time: 0.001013
step_time: 0.000799


step_time: 0.003905
step_time: 0.001041


step_time: 0.001056
step_time: 0.000972
step_time: 0.001015


step_time: 0.000767
step_time: 0.000824
step_time: 0.000598


step_time: 0.000797
step_time: 0.000793
step_time: 0.000781


step_time: 0.000599
step_time: 0.000124
step_time: 0.000119
step_time: 0.000062

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 27) ---
  Sol: TATCACGTAGGATACTGAATCCTAAAC
str01: -AT---G--GGATAC-G----------
str02: -AT-AC--------CT---TCC----C
str03: ---CACG-A--AT--TGA---------
str04: TA--A---A--AT-CTG--T-------
str05: -A----G--G--TA---A--C--AAA-
str06: T-TC-C-TAGG-TA-------------
str07: T-T---GTAG-AT-CT-----------
str08: T-----G--GGA-A--G--T--T---C
str09: T-TC-C--A-----C--AA-C-T----
str10: T--C---TA--A-AC-GAA--------

wall time: 0.143077s
solution is feasible: True


In [ ]:
bench1(scsp.example.load("nucleotide_n050k050.txt"))

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

T--G--
str21: A-GAGC--A---A--TC-A--G--T---G-C-ATC-AG--A----A-AT-AT-AC-C-TA---T---T-A---T-A---C-A--C-T---T----TGCTA--A-G-A-----A---T---------------------------
str22: A--A--T----TA-A---A-A-CATC-T--C-A---A--TA-C--A-A-CAT-A-A-G-A-A--A----AA-C--A-A-CG---CA--A--A--A----A-CA--C--T-C-A---T---------------------------
str23: A--A----A-C--GA---A---C-T--T-T--A---A---A----A--TC-TG-----T---GT-G-----GCTG-T--C-A--C-T--C--G---GCT-GCATGC--T--TA--GTGC-------------------------
str24: AT-A----A-CTA-AT---TA-C-T---GTCG-T-T-G--A-C--AG-----GACACG-A--GTA----A--CT-----CG-T-C-T-A-T--C-T--T--C-TG---------------------------------------
str25: ATGAG-T--G-T----C-A---C-----G---A---A--T----T----CA---C--GTACA--A--T---G---A-A-C--TG---GA-T-G--T--T--CA--C---G-T---G-G--A--AT-A---A-------------
str26: A----C-C-G-T-G---G---GC-----G---A----GC--G-GT-GA-C----C--G----GT-G-TC----T--T--C----C-T-A---G--TG---G---G---T-C--C----C-ACG-T----T-G--AA----R---
str27: A--A----AG---G-T---T----T-AT----A-C---CT----T----C----C-C--A--G--G-T-AA-C-

G---G-T----TG---T-C-AC--T-C-------------------------
str42: A--A-C-CA---A---C-----CA--A---C--T-T---T--CG-A--TC-T--C---T----T-G-T-A-G---AT--C--TG--T---T--C-T-CTA--A---AC-G--A-A---CT----T----TA-------------
str43: --G-G----G-T---TC--T-GC--CA-G--G--C-A--TAG--T----C-T------T----T---T-----T--T-TC--TG---G-C--G---GC---C---C--T--T---GTG-TA--A--A-C----C----T--G--
str44: --G-GCT--GC-A--T-G----C-T--T----A----G-T-GC--A---C-T--CACG--CAGTA--T-AA--T--TA---AT--A--ACTA--AT--TA-C-TG---T-----------------------------------
str45: -TG--C--A--T-G--C--T----T-A-GT-G--C-A-CT--C--A---C--G-CA-GTA---TA----A---T--TA---AT--A--ACTA--AT--TA-C-TG---T-C----GT---------------------------
str46: -T----TC--C-A---C-A-A-C-T--T-TC---C-A-C---C--A-A----G-C---T-C--T-GC--AAG---AT--C----C----C-AG-A-G-T--CA-G----G-----G-GC--C--T--G-T--------------
str47: -T---CT-A---A-A-CGA-A-C-T--T-T--A---A---A----A--TC-TG-----T---GT-G-----GCTG-T--C-A--C-T--C--G---GCT-GCATGC--T--TA--G----------------------------
str48: A----C-C-G---GAT-G---GC--C--

In [ ]:
bench2(scsp.example.load("nucleotide_n050k050.txt"))

step_time: 0.002234


step_time: 0.002013


step_time: 0.001952


step_time: 0.002128


step_time: 0.002434


...

step_time: 0.000230


step_time: 0.000166


step_time: 0.000114

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTG

-CT--G----GC-----C--C----C------G---G--G--C----A--A--A--T-G--C---C---CT-A--A--T---C-CA-G-AG-G--T--G-
str21: A-GAGC--A---A-TCA-G---TGC-ATC-A-G--A--A---A-TA---T-A-C-----CT--AT-------TA-T-A--C-A-C-T---T---T-GCTA-A---GA--A--T-----------------------------
str22: A--A--T---T-A---A--A-A--C-ATC--T-C-A--A-T-AC-A--A-CA--TA--A--G-A-A---AA--A--CA----A-CG--CA--A--A---A-A-C--ACT--C--A-T-------------------------
str23: A--A----A--C-G--A--AC-T--T-T--A----A--A---A-T-C--T--G-T-GT---G----GC----T-GTCA--C--TCG-GC-TG-C-A--T-G--CT---TAG-TG-C--------------------------
str24: AT-A----A--C--T-A--A--T--TA-C--TG-T--C-GT---T--GA-CAG---G-AC---A---C------G--AGT--A------A---CT--C--G-TCT-A-T--CT---T--CT-G-------------------
str25: ATGAG-T--GTCA--C--GA-AT--T--C-A--C--G---T-AC-A--AT--G--A--ACTG----G--A--T-GT---TC-A-CGTG---GA--A--TA-A----------------------------------------
str26: A----C-C-GT--G----G----GC----GA-GC--G--GTGAC--CG----G-T-GT-CT---T--CC---TAGT--G--G---GT-C----C---C-A---C-G--T---TGA--A--------------------R---

-CTA--A---AC---GA--A-CT--T--T--A-A---AA-T---C--T-G-T-GTG---G--T---T-G-TC--ACT--C------------------------------
str42: A--A-C-CA---A--C----CA----A-C--T--T-----T--C---GATC---T----CT---T-G-----TAG--A-TC--T-GT---T--CT--CTA-A----AC--G---A--A-CT--T----T--A----------
str43: --G-G----GT---TC-TG-C---C-A--G--GC-A----T-A----G-TC---T--T--T---T-------T--T---TC--T-G-GC--G----GC-----C---CT---TG--T-G-TA---A-A--C----C-T--G-
str44: --G-GCT--G-CA-T---G-C-T--TA--G-TGC-A-C--T--C-ACG--CAG-TA-TA----AT-------TA---A-T--A------A---CTA---A--T-T-ACT-G-T-----------------------------
str45: -TG--C--A-T--G-C-T----T---A--G-TGC-A-C--T--C-ACG--CAG-TA-TA----AT-------TA---A-T--A------A---CTA---A--T-T-ACT-G-T--C--G-T---------------------
str46: -T----TC---CA--CA--AC-T--T-TC----C-A-C-----C-A--A---GCT----CTGCA-AG--A--T---C---C---C----A-GA---G-T----C--A---G--G----G---G-C-C-TG-------T----
str47: -T---CT-A---A---A---C--G--A---A--CT-----T---TA--A--A---A-T-CTG--T-G-----T-G---G-C--T-GT-CA---CT--C--G----G-CT-GC--A-T-GCT--T-A---G--

In [ ]:
bench1(scsp.example.load("protein_n010k010.txt"))

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 51) ---
  Sol: MEQSKAFPLSVEYACHPGFLRVNTAEQHFRDAIKGGRALHDNPSRNGTVYQ
str01: M----A--LS--Y-C-P----------------KG------------T---
str02: M-QS-----S---------L--N-A-------I---------P-----V--
str03: M------PLS--Y-------------QHFR---K-----------------
str04: ME---------E---H-----VN--E------------LHD----------
str05: M--S------------------N-----F-DAI---RAL------------
str06: M-----F-------------R-N---Q--------------N-SRNG----
str07: M-----F-----YA-H--------A---F-----GG-------------Y-
str08: M--SK-F----------------T-----R------R-----P------YQ
str09: M--S--F---V--A---G---V-TA-Q------------------------
str10: ME-S----L-V-----PGF---N--E-------------------------

wall time: 0.034474s
solution is feasible: True


In [ ]:
bench2(scsp.example.load("protein_n010k010.txt"))

step_time: 0.004159


step_time: 0.002024


step_time: 0.002008


step_time: 0.002457


step_time: 0.002220


...

step_time: 0.000891


step_time: 0.000879


step_time: 0.000139


step_time: 0.000122


step_time: 0.000060

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 51) ---
  Sol: MESKFALVPLSYAGQEHVFTRNAEQNSSRLNFCPKDAYQIGGHDPRALTVY
str01: M----AL---SY--------------------CPK-----G-------T--
str02: M-------------Q-----------SS-LN-----A--I----P----V-
str03: M-------PLSY--Q-H-F-R-------------K----------------
str04: ME-------------EHV---N-E-----L------------HD-------
str05: M-S------------------N---------F---DA--I-----RAL---
str06: M---F---------------RN--QNS-R-N---------G----------
str07: M---F------YA---H-----A--------F--------GG--------Y
str08: M-SKF--------------TR-------R----P---YQ------------
str09: M-S-F--V----AG---V-T--A-Q--------------------------
str10: MES---LVP----G----F--N-E---------------------------

wall time: 0.864689s
solution is feasible: True


In [ ]:
bench1(scsp.example.load("protein_n050k050.txt"))

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

-------------------V-----------------------------------------
str09: M--------------N--------T-------------------G-------------I-------------ID--L----------F-----D-N---------------------------------------H------VD----S-IP----TI-L-------P---------------H-QL----A--T------LD-----Y----L-------VR---T-I-------I--D--E-N--R-------S-----V-----------L----------------L------F--------------------------H---------I---------------------M------G--S-------------G--------------------------------------------------------------------------------------------------------------------
str10: M-FV-F-LVLLP--L--V--SSQC--VN----L------RT-----R---TQ--LP-----------P--A---------YTN---SF--T-R-G-----V-----------Y-------Y-PD-K-VF---R----S--S-V-L--HS-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

-
str15: M---------------R---------------------------------------Y-IV---S---P-Q------L------V-L------------Q-V--------------------G---K--------G----Q-------------E-----------V-------E-------RA---L---Y-L-TP------------Y---------D------------------------Y---------I----D-E-K--------S---------------P-----I------------Y--------------------------------Y--F-----L--R---S------------------HL----------------N-I-------------QRP--------------------------------------------------------------------------------------
str16: M----------P----RV------------P----------V--------------Y----------------D-----S------------------------P----------------------------Q--------V-----S--P---NT--------V-P-----------------Q-----A----R----L--A--T--------P---S--F----------A------T---P-----------------TF-R-----G-------A-D---------A-------P--AF--QD---------T-------A--N--------Q-------------------------------QA------R---------Q-------------------------------------------------------------------------------------------------

----L-A----E------------G------F----Q---------------------------R----Y-------N-K-----------------TP-G-F-------T------------------------------------------------------------C--------------------VL-------DR------Y-D---------------H--G--V---I-----------------N-DS--KIVL------YN-------------------------------------------------------------------------------------------------------------------------------
str24: M------------------K-------NI----A---------------E---------------FK-----------------------------K------AP---E----------L----------A----------------------E----KL------------LE-----VF------S---------------------------------------N---------LK-G---N----------S----------R----S-L--------D----P-----------------------------M-------RAG----K-------------H--D----------V--------------------V-----------VIE--STK--K---L-----------------------------------------------------------------------------------------
str25: M----------P----------Q-------P-L--------------------K---------------Q---------S-----L-

N-----F-------------T-RG--V-------Y-----Y-P-DK-V-F-----R-SSV---L--------H---L------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str32: M------------------------H-----Q-----I--TV---V--S-------------G----PT-----E--------V--S---T----------------C--------F----G----S---------L----------H---P--F-----Q--S--------L---K------------------P----V----M-----------A---------N------A--L--G-------V-LEG-K--------------------------M-F----------C-S--------I-----G----G--------R-------S-L-----------------------------------------------------------------------------------------------------------------------------------------------------------------
str33: MA----------------------T-------L---L--R--------S-----L--A------LFK--------R------N-------------K------------D---K--------P----------------------------P-----I------

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str40: M---E-SLV--PGF-N--EK----THV----QL--SL-P--VLQ-V-------------------------------------------------------------C-D-------V-L-------V----R-G-------------------F---------G---D--------S-V----E-------------E-VL----S---E------A----R----------------------------------------------------QH-------------LK------D--GT--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str41: M--------------N-----------N---Q-------R-------------K------------K-T-A----R--PS-------F-------N-M---L-----------K------------------R-------------A--------------RN------------------R------------------V-----

---I---------------------------------------------------------------W----T---S--------L----G--------------------------------------F----------------------------------------W-M-FMIC--SV-VGN-L--WVTVYYGVPVWKEAKTT
str48: MA-VE------P-F----------------P--------R------R--------P--I---------T------R--P---------HA---------------SI-E--------V-----D-------T-----S-----------GI-------------G----------G-SA------------------G--------S----S------------E-------------K---------V---------------F---C----L---I--------G--------------------Q-----AE-G----------G-----------------E-------P---------------N--------------T--------V-------------------------------------------------------------------------------------------------------
str49: M-F-----------------------------------------------------YA------------------------------HA--------------------------F----G------------G-----------------Y---------------D----EN-----------L----------------HA----------FP--------G--I--SS--------T------VA------N-D--V----R-------------

In [ ]:
bench2(scsp.example.load("protein_n050k050.txt"))

step_time: 0.005662


step_time: 0.022808


step_time: 0.024436


step_time: 0.029126


step_time: 0.022653


...

--A-------LA-EG---F-------Q-R-----------Y--NK-------T---------PG-------F---T-------------------C--V---------L-----------D---------------R-----Y------------D------------H--G----V------------------I----N----------DS----K-IV-L----Y---------N--------------------------------------------------------------------------------------------------------------
str24: M---------------K-N-----------I------------------------A-----------------------E---------F------------------K------------------K-----A---------------------------P-----ELA-----------------E--KL--------L--E-----V--F------------S----------------------NL----K--------------G-------------N-----S---------RSLD---P-----------------------------MRA------------------G--K------------------H--D-----V------------------VV----------I---ESTK-----K----L--------------------------------
str25: M----------P--------Q----------------P-L----------K---------------Q---------S-----------L------------------D-----Q--S----------K--------------------------------------

-M--D---------------S-------------------------------------------------C-----------------------------------CL-IS---RI-----T---P----------E-L-------A-------GK-LTW----I----------F-------I-------------------------------
str30: M----ESLV--P---G-FN-----E--K---THVQ-L-SL--PVL-Q-V---R-D---V-LVR----GF---G-D-SV-E------E--F-------------L------------SE------A-------------------------R-------Q----------------H--L-----------K--------------DGT----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str31: M-FVF--LVLLPLVS----SQ-CV------------------------------------------------------M------P--LF---N---------L----------------I--------T---------------T-------T----Q-S-Y-TN------F-T----RGVYYP-D---K-VFR---S---------SV---L-----------------H-----------------L-------------------------------------------------

--------------------------------------------------------------------------
str36: MA----------------N-----------I--------------------------I-NL-------------------------------WN-GI--V------P-----------------------------------M----VQD---------V-----N---------------V--------------A-S----------------------I------TA-------F---K-S------------MI-D------E---T-----WDK-------------K----I---------E-----A--------N-TC--I-----S--R-K-----------H---R---N--------------------------------------------------------------------------------------------------------------
str37: M------L----------N--R--------I---Q------T--L---------------------------------M-----------------------------K---T-----------A----------------N---------------N----Y----E------T--I---------E-I-L--RN------Y----------LR---------------L--------------Y-----------I--------------I-L----AR--N-E---------E---------------G------RG--------I-L----------I-------Y--D---D--N----I--D-------S------------V---------------------------------------------------

----------D----T-D---------A-YK----L---G--------------------------------------H-I--------------------------H---------------------------------------------------M-------YPE--GT--------------E-----------------------------Y----V---------L---S-----N--F---T----D--R-G-------S---RIEGV--T--H-------TV-------------H-------------------------------------------------------------------
str43: M-----------------------------I--------------------E--------L-R---------------------------------------H--------------E---V---------Q----GD------L--V-----TI--N-V---------------------V-----E---------T---P-E-D-------L---------DG------------FR---D------------F-IR--AHL--------ICL----A-V-----DT------E------------------T---------T----GL--D-------I-------Y------------------------------------------------------------------------------------------------------------------------
str44: M-FVF--LVLLPLVS----SQ-CV------------------------------------------------------M------P--LF---N---------L----------------I--------T-----------

-YQ--------------S--------NS--Y---------V--E-L--R-------RN----------------Q-----------A--L---------N-K-------NL-----------

wall time: 48.255862s
solution is feasible: True


DIDP を使うと時間が長くなる.
小さな問題であれば直接プログラミングした方がよい.